### Import Modules

In [2]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

matplotlib.rcParams['figure.figsize'] = [10, 10]

In [57]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Specify fit parameters

In [58]:
p_dict = {
    'order' : {
        'fit' : 'nlo', # 'nlo', 'nnlo', or 'nnnlo'
        'latt_spacing' : 2, # no order 1 term -- starts at 2
        'vol' : 1
    }, 
    'bs_N' : 3,  # if 0, use full list
    'fit_type' : 'ma-taylor', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'
    
    'use_prior' : False,
    'save_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350',
           u'a15m400'],
    
    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False
}

### Do fit

In [59]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior()

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

if p_dict['make_plots']:
    data_loader.save_plots(bootstrapper.make_plots(
        show_error_ellipses=p_dict['show_error_ellipses'], show_bootstrap_histograms=p_dict['show_bs_histograms']))
else:
    print bootstrapper
    
if p_dict['save_prior']:
    print 'Saving prior...'
    data_loader.save_prior(bootstrapper.create_prior_from_fit())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Making fits...
100% complete  Time (s):  29.6499998569
Compiling results...
Done.

Total time (s):  31.5570001602 



In [60]:
print bootstrapper.fits[0]

Least Square Fit:
  chi2/dof [dof] = 2.5e+06 [14]    Q = 0    logGBF = -1.7486e+07

Parameters:
          Fpi 0    0.085014 (12)       [  0.042843 (52) ]  *****
              1    0.069255 (32)       [  0.045523 (39) ]  *****
              2    0.081117 (14)       [  0.046632 (21) ]  *****
              3   0.0495960 (17)       [ 0.0483683 (18) ]  *****
              4    0.105016 (21)       [  0.057038 (51) ]  *****
              5    0.093115 (15)       [  0.058695 (19) ]  *****
              6    0.108252 (43)       [  0.058650 (99) ]  *****
              7    0.067510 (46)       [  0.061385 (48) ]  *****
              8    0.107274 (64)       [  0.062992 (64) ]  *****
              9    0.064998 (67)       [  0.064984 (67) ]  
             10    0.080635 (17)       [  0.072768 (17) ]  *****
             11    0.143983 (94)       [   0.07529 (16) ]  *****
             12    0.111825 (77)       [  0.076901 (77) ]  *****
             13     0.14015 (15)       [   0.07938 (21) ]  *****

In [35]:
print bootstrapper



Fitting to nlo 
 with lattice corrections O(2) 
 with volume corrections O(1) 
Fitted/[Experimental] values at physical point:

F_K / F_pi = 0(0) [1.201(46)]

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
    c_a 0,0              17.8 (5.3)            1               1.0 (1.0)
        0,1                  2 (10)        L_4 0           ['0.38 (62)']
        1,0               0.0 (1.0)            1               1.0 (1.0)
        1,1               0.0 (1.0)        L_5 0           ['0.35 (65)']
  c_mpia2 0           ['2.6 (1.7)']            1               1.0 (1.0)
---
boot0 fit results:
Least Square Fit:
  chi2/dof [dof] = 1.7e+06 [14]    Q = 0    logGBF = -1.157e+07

Settings:
  svdcut/n = 1e-12/15    tol = (1e-08,1e-10,1e-10)    (itns/time = 1000*/11.6)
  fitter = scipy_least_squares    method = trf

